In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio
import json

# Import your modified run_finance_agent function
from your_finance_agent_module import run_finance_agent

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QueryInput(BaseModel):
    query: str

async def event_generator(query):
    async for state in run_finance_agent(query):
        yield f"data: {json.dumps(state)}\n\n"
    yield "data: [DONE]\n\n"

@app.post("/query")
async def process_query(query_input: QueryInput):
    return StreamingResponse(event_generator(query_input.query), media_type="text/event-stream")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
import streamlit as st
import asyncio
import sys
from typing import Dict, Any

# Import your updated finance agent function
from your_finance_agent_module import run_finance_agent

st.title("Finance Agent Query Interface")

query = st.text_input("Enter your query:")

if st.button("Submit"):
    if query:
        st.write(f"Processing query: {query}")
        
        graph_placeholder = st.empty()
        results_placeholder = st.empty()
        final_response_placeholder = st.empty()

        def update_graph(current_node: str):
            nodes = ["subgraph_router", "execute_subgraphs", "aggregate_results", "end"]
            graph = " -> ".join([f"[{node}]" if node == current_node else node for node in nodes])
            graph_placeholder.text(f"Current State: {graph}")

        async def run_query():
            try:
                async for state in run_finance_agent(query):
                    st.write(f"Received state: {state['current_node']}")
                    update_graph(state['current_node'])
                    
                    if 'state' in state:
                        results_placeholder.json(state['state'])
                    
                    if state['current_node'] == 'end':
                        final_response = state.get('final_response', 'No final response available.')
                        final_response_placeholder.markdown(f"**Final Response:**\n\n{final_response}")
                
                st.success("Query processing completed!")
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")
                st.exception(e)

        asyncio.run(run_query())
    else:
        st.warning("Please enter a query.")

st.markdown("""
## How to use this app:
1. Enter your finance-related query in the text box above.
2. Click the 'Submit' button to process your query.
3. Watch the graph update to show the current state of processing.
4. View the detailed results and final response below the graph.
""")

st.subheader("Debug Information")
st.write(f"Streamlit version: {st.__version__}")
st.write(f"Python version: {sys.version}")

In [ ]:
import streamlit as st
import asyncio
import sys
from typing import Dict, Any

# Import your updated finance agent function
from your_finance_agent_module import run_finance_agent

st.title("Finance Agent Query Interface")

query = st.text_input("Enter your query:")

if st.button("Submit"):
    if query:
        st.write(f"Processing query: {query}")
        
        graph_placeholder = st.empty()
        results_placeholder = st.empty()
        final_response_placeholder = st.empty()

        def update_graph(current_node: str):
            nodes = ["subgraph_router", "execute_subgraphs", "aggregate_results", "end"]
            graph = " -> ".join([f"[{node}]" if node == current_node else node for node in nodes])
            graph_placeholder.text(f"Current State: {graph}")

        async def run_query():
            try:
                async for state in run_finance_agent(query):
                    current_node = state.get('current_node', 'unknown')
                    update_graph(current_node)
                    
                    # Display the state
                    display_state = state.get('state', {})
                    results_placeholder.json(display_state)
                    
                    if current_node == 'end':
                        final_response = display_state.get('final_response', 'No final response available.')
                        final_response_placeholder.markdown(f"**Final Response:**\n\n{final_response}")
                
                st.success("Query processing completed!")
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")
                st.exception(e)

        asyncio.run(run_query())
    else:
        st.warning("Please enter a query.")

st.markdown("""
## How to use this app:
1. Enter your finance-related query in the text box above.
2. Click the 'Submit' button to process your query.
3. Watch the graph update to show the current state of processing.
4. View the detailed results and final response below the graph.
""")

st.subheader("Debug Information")
st.write(f"Streamlit version: {st.__version__}")
st.write(f"Python version: {sys.version}")

In [ ]:
# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def run_finance_agent(query: str) -> AsyncGenerator[Dict[str, Any], None]:
    logger.info(f"Starting run_finance_agent with query: {query}")
    graph = create_finance_agent_graph()
    logger.debug(f"Created graph: {graph}")

    initial_state = AgentState(
        query=query,
        expanded_queries=[
            "What are the recent internal views on Apple Inc's relationship with its suppliers?",
            "How have Apple Inc's suppliers impacted its business operations recently?",
            "What are the latest developments in the partnerships between Apple Inc and its suppliers?"
        ],
        subgraphs_to_execute={},
        subgraph_results={},
        final_response=""
    )
    logger.debug(f"Initial state: {initial_state}")

    try:
        result = await graph.ainvoke(initial_state)
        logger.debug(f"Result from graph.ainvoke: {pformat(result)}")

        # Check if result is iterable
        if hasattr(result, '__iter__') or hasattr(result, '__aiter__'):
            async for state in result:
                logger.debug(f"Received state from result iteration: {pformat(state)}")
                yield await process_state(state)
        else:
            # If result is not iterable, process it as a single state
            logger.debug("Result is not iterable, processing as single state")
            yield await process_state(result)

        logger.info("Graph processing completed")
    except Exception as e:
        logger.exception(f"An error occurred during graph processing: {e}")
        raise

async def process_state(state):
    # Attempt to extract current_node information
    current_node = "unknown"
    if hasattr(state, 'current_node'):
        current_node = state.current_node
    elif isinstance(state, dict) and 'current_node' in state:
        current_node = state['current_node']
    
    logger.info(f"Current node: {current_node}")
    
    # Debug print of all attributes/items in state
    if isinstance(state, dict):
        for key, value in state.items():
            logger.debug(f"State[{key}]: {value}")
    else:
        for attr in dir(state):
            if not attr.startswith('__'):
                logger.debug(f"State.{attr}: {getattr(state, attr)}")
    
    return {
        "current_node": current_node,
        "state": state
    }

async def process_query(query: str):
    logger.info(f"Processing query: {query}")
    try:
        async for state in run_finance_agent(query):
            current_node = state.get('current_node', 'unknown')
            logger.info(f"Current node: {current_node}")
            
            display_state = state.get('state', {})
            logger.debug(f"State: {pformat(display_state)}")
            
            if current_node == 'end':
                if isinstance(display_state, dict):
                    final_response = display_state.get('final_response', 'No final response available.')
                else:
                    final_response = getattr(display_state, 'final_response', 'No final response available.')
                logger.info(f"Final Response: {final_response}")
        
        logger.info("Query processing completed!")
    except Exception as e:
        logger.exception(f"An error occurred during query processing: {e}")